<a href="https://colab.research.google.com/github/akshithamaddi/akshithamaddi_INFO5731/blob/master/Bertfortweetswithn_grams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
#topic Modeling
import pandas as pd
topicm = pd.read_csv("2020-3-new.csv",encoding="ISO-8859-1")
header = ["clean_tweet_text"]
topicm.to_csv('output.csv', columns = header)

In [16]:
pip install pyLDAvis==2.1.2

     |████████████████████████████████| 1.6MB 5.6MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=5247fb73f079a17b8580a933c06f569d03e4f5d0d6cc014151e652542ede94e4
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


In [17]:
pip install "gensim==3.8.1"

     |████████████████████████████████| 24.2MB 163kB/s 
ERROR: flair 0.7 has requirement sentencepiece<=0.1.91, but you'll have sentencepiece 0.1.95 which is incompatible.
ERROR: flair 0.7 has requirement transformers<=3.5.1,>=3.5.0, but you'll have transformers 4.5.0 which is incompatible.
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [1]:
import spacy.cli
spacy.cli.download("en_core_web_sm")

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [2]:
import nltk;
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [11]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [12]:
data = topicm.clean_tweet_text.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

print(data[:1])

['sources telling early friday USER_MENTION ask employees lie home chinavirus chinesevirus indiafightscoronavirus']


In [13]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['sources', 'telling', 'early', 'friday', 'user_mention', 'ask', 'employees', 'lie', 'home', 'chinavirus', 'chinesevirus']]


In [14]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['sources', 'telling', 'early', 'friday', 'user_mention', 'ask', 'employees', 'lie', 'home', 'chinavirus', 'chinesevirus']


In [15]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [16]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['source', 'tell', 'early', 'ask', 'employee', 'lie', 'home', 'chinavirus']]


In [17]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]]


In [18]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('ask', 1),
  ('chinavirus', 1),
  ('early', 1),
  ('employee', 1),
  ('home', 1),
  ('lie', 1),
  ('source', 1),
  ('tell', 1)]]

In [20]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=38, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [21]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(16,
  '0.000*"wwaayy" + 0.000*"lawrence" + 0.000*"joebiden" + 0.000*"outflank" + '
  '0.000*"whosoever" + 0.000*"trumpandemic" + 0.000*"taxation" + '
  '0.000*"lifethreatene" + 0.000*"berniesander" + 0.000*"container"'),
 (26,
  '0.000*"wwaayy" + 0.000*"lawrence" + 0.000*"joebiden" + 0.000*"outflank" + '
  '0.000*"whosoever" + 0.000*"trumpandemic" + 0.000*"taxation" + '
  '0.000*"lifethreatene" + 0.000*"berniesander" + 0.000*"container"'),
 (23,
  '0.000*"wwaayy" + 0.000*"lawrence" + 0.000*"joebiden" + 0.000*"outflank" + '
  '0.000*"whosoever" + 0.000*"trumpandemic" + 0.000*"taxation" + '
  '0.000*"lifethreatene" + 0.000*"berniesander" + 0.000*"container"'),
 (7,
  '0.000*"wwaayy" + 0.000*"lawrence" + 0.000*"joebiden" + 0.000*"outflank" + '
  '0.000*"whosoever" + 0.000*"trumpandemic" + 0.000*"taxation" + '
  '0.000*"lifethreatene" + 0.000*"berniesander" + 0.000*"container"'),
 (2,
  '0.000*"wwaayy" + 0.000*"lawrence" + 0.000*"joebiden" + 0.000*"outflank" + '
  '0.000*"whosoever" + 0.

In [ ]:
coherence = []
perplexity = []
for k in range(5,75):
  print('Round: '+str(k))
  lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=k, 
                                           chunksize=10000,
                                           passes=10,
                                           iterations=200,
                                          eval_every = None) 
  coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')                                              
  coherence.append([k,coherence_model_lda.get_coherence()])
  doc_lda = lda_model[corpus]
  perplexity.append(lda_model.log_perplexity(corpus))

In [ ]:
print(coherence)
print(perplexity)

[[5, 0.34987340537719935], [6, 0.3140970390893331], [7, 0.30507482771884853], [8, 0.3335139438150748], [9, 0.34754868298642067], [10, 0.37886358959704936], [11, 0.34323949101170054], [12, 0.3586346535462459], [13, 0.3624084517384147], [14, 0.3563272054975001], [15, 0.3664047568169619], [16, 0.34771817288818574], [17, 0.3653916947241706], [18, 0.33826525306672445], [19, 0.37896852942726433], [20, 0.3829741843081077], [21, 0.38978701689730016], [22, 0.36480234767479036], [23, 0.3772903441634384], [24, 0.3764081043428564], [25, 0.38988774409395477], [26, 0.39002701290529623], [27, 0.4000362447810887], [28, 0.3899791856721394], [29, 0.38197406770859443], [30, 0.38473864482978926], [31, 0.4023445432567965], [32, 0.4051956589600385], [33, 0.41796591751033346], [34, 0.41555616084557867], [35, 0.41512456899761196], [36, 0.41929056175156576], [37, 0.4038544424946787], [38, 0.4327979527706474], [39, 0.44497599016415573], [40, 0.43754315445588643], [41, 0.42691854513963845], [42, 0.44085876813972

In [ ]:
column_names = ["num topics","Coherence"]
df = pd.DataFrame(coherence, columns=column_names)
df['perplexity'] = perplexity
df

,num topics,Coherence,perplexity
0,5,0.349873,-6.438674
1,6,0.314097,-6.451023
2,7,0.305075,-6.391372
3,8,0.333514,-6.481422
4,9,0.347549,-6.522799
...,...,...,...
67,72,0.512050,-16.892637
68,73,0.495329,-17.023939
69,74,0.512100,-17.301782
70,75,0.484277,-17.314453


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=df['num topics'], y=df['Coherence'], name="coherence"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df['num topics'], y=df['perplexity'], name="perplexity"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Coherence and preplexity"
)

# Set x-axis title
fig.update_xaxes(title_text="Num topics")

# Set y-axes titles
fig.update_yaxes(title_text="Coherence", secondary_y=False)
fig.update_yaxes(title_text="Perplexity", secondary_y=True)

fig.show()

In [23]:
import pandas as pd
df = pd.read_csv("2020-3-new.csv",encoding="ISO-8859-1")
header = ["clean_tweet_text"]
df.to_csv('output.csv', columns = header)

In [24]:
pip install bertopic

In [25]:
pip install bertopic[visualization]

In [26]:
pip install bertopic[flair]

In [27]:
from bertopic import BERTopic

df1= pd.read_csv("output.csv")
docs = df1['clean_tweet_text']
topic_model = BERTopic()
topics, probabilities = topic_model.fit_transform(docs)

In [28]:
new_topics, new_probs = topic_model.reduce_topics(docs, topics, probabilities, nr_topics=38)

In [ ]:
topic_model.get_topics()

{-1: [('covid19', 0.016650093473956604),
  ('coronavirus', 0.01373414803035172),
  ('wuhanvirus', 0.012767122327502952),
  ('coronavirusoutbreak', 0.012047448566644353),
  ('covid2019', 0.011836409167569704),
  ('quarantinelife', 0.00868337406473148),
  ('corona', 0.008490167208637045),
  ('chinaliedpeopledied', 0.006648136626478216),
  ('wuhancoronavirus', 0.006457812810471358),
  ('socialdistancing', 0.006236408033187403)],
 0: [('afraid', 0.12193361694433838),
  ('journalists', 0.12113263521124543),
  ('propaganda', 0.10402103355841139),
  ('muttering', 0.0004843938430637791),
  ('complymost', 0.0004843938430637791),
  ('backstab', 0.0004843938430637791),
  ('india', 0.00047731172753200335),
  ('trap', 0.0003944311301434361),
  ('consumer', 0.00039146668713258204),
  ('market', 0.00025669561485574144)],
 18: [('karimnagar', 0.19627739793318477),
  ('ranch', 0.19608853216765898),
  ('erode', 0.19583775398992392),
  ('smoking', 0.19565044634240006),
  ('pain', 0.18865434603393788),
  

In [29]:
topic_model.visualize_topics()

In [30]:
#tri-grams
topic_model.update_topics(docs, topics, n_gram_range=(1, 3))

In [31]:
topic_model.get_topics()

{-1: [('coronavirusinpakistan', 0.0006455201828020146),
  ('coronaviruspandemic', 0.0006135502729638804),
  ('chinesevirus coronavirusoutbreak', 0.0006082918363297201),
  ('covid2019 chinesevirus', 0.0005569671199348878),
  ('coronavirus coronavirusoutbreak', 0.0005461663432487919),
  ('covid2019 chinavirus', 0.0005415223462163189),
  ('chinesevirus covid2019', 0.0005411459107399339),
  ('coronavirusindia', 0.0005282933129715014),
  ('coronavirus covid19', 0.0005053384437981027),
  ('chinesevirus chinesecoronavirus', 0.0005030131989169876)],
 8: [('thats chinesevirus looks', 0.16029357740844694),
  ('url thats chinesevirus', 0.16029357740844694),
  ('chinesevirus looks like', 0.15967928952682764),
  ('chinesevirus looks', 0.1590731381480388),
  ('thats chinesevirus', 0.15288703202605322),
  ('coronapocalyse', 0.008798452927595837),
  ('coronapocalyse chinavirus', 0.008798452927595837),
  ('coronapocalyse chinavirus url', 0.008798452927595837),
  ('like coronapocalyse chinavirus', 0.008

In [32]:
#bi-grams
topic_model.update_topics(docs, topics, n_gram_range=(1, 2))

In [33]:
topic_model.get_topics()

{-1: [('coronavirusinpakistan', 0.0007472509415917297),
  ('coronaviruspandemic', 0.0006950458622261058),
  ('chinesevirus coronavirusoutbreak', 0.0006786573159974596),
  ('covid2019 chinavirus', 0.0006698886329124671),
  ('chinaliedpeopledied chinaliedpeopledied', 0.0006603453977298685),
  ('islamicrepublicvirus', 0.0006360357071000547),
  ('coronavirusoutbreak chinesevirus', 0.0006051063749631773),
  ('coronavirus coronavirusoutbreak', 0.0006024714309210174),
  ('chinesevirus chinesecoronavirus', 0.0005993738161627407),
  ('chinesevirus chinaliedandpeopledied', 0.000588935888651727)],
 8: [('chinesevirus looks', 0.2106228078274837),
  ('thats chinesevirus', 0.20135224842131993),
  ('corona url', 0.15225606497259417),
  ('like coronapocalyse', 0.012435022054094318),
  ('coronapocalyse', 0.012435022054094318),
  ('coronapocalyse chinavirus', 0.012435022054094318),
  ('url thought', 0.0036479199962410596),
  ('everything scanned', 0.0),
  ('everything mistake', 0.0),
  ('everything mass